# Análisis de datos de Webscraping

Tomando de base el output.xlsx sacado con el cuaderno de scraping_login vamos a intentar hacer un análisis de datos básico.

## Estructura:

La estructura del fichero es la siguiente:

id = id de la url e identificador del usuario

username = nombre del usuario, obligatorio sino la fila no estaría

profile_pic = url a la foto de perfil, si no hay será "No photo"

create_date = fecha de primer acceso tras la creación del usuario, puede ser "Nunca" si no se ha accedido por primera vez o "N/A" si hubo error de lectura, no viene en formato estandar

last_conn = fecha de ultima hora de conexión, puede ser "Nunca" si no se ha accedido por primera vez o "N/A" si hubo error de lectura, no viene en formato estandar

email = direccion de correo, en caso de no haber será "No email"

courses = array de strings de los cursos a los que está inscrito el usuario, puede ser vacío, los strings vienen con sigle comma ('ejemplo')



Vamos a empezar por cargar el Excel a un DataFrame

In [1]:
from pathlib import Path
import pandas as pd

xlsx_path = Path.cwd() / "output.xlsx"

# Leer Excel
df = pd.read_excel(xlsx_path)

# Mostrar las primeras filas
print(df.head(3).to_string(index=False))

 id       username                                                                  profile_pic                                               create_date                                                     last_conn    email courses
  1     Gonbidatua                                                                     No photo                                                     Nunca                                                         Nunca No email      []
  2 Admin ICJardin                                                                     No photo       viernes, 5 de julio de 2024, 09:55  (1 año 94 días) martes, 7 de octubre de 2025, 10:12  (10 minutos 14 segundos) No email      []
  3     Admin User https://moodle.icjardin.com/pluginfile.php/18/user/icon/boost/f1?rev=2622497 martes, 17 de noviembre de 2020, 09:45  (4 años 324 días)          jueves, 27 de junio de 2024, 19:45  (1 año 101 días) No email      []


Quiero ver algunas estadísticas:

In [2]:
# Muestra (filas, columnas) en formato tupla
print(df.shape)

# Guardamos la cantidad de filas
total_rows = df.shape[0]

# Calculamos cuanta gente tiene foto creando un subset de DataFrame
with_photo = total_rows - df[df["profile_pic"] == "No photo"].shape[0] # Total - gente sin foto
print(f"Cantidad de gente con foto: {with_photo}, eso es el {((with_photo*100)/total_rows):.2f}% de los usuarios")

# Lo mismo con emails, probamos a usar el pseudoSQL de pandas con .query y simplificamos el cálculo
with_email = df.query("email != 'No email'").shape[0]
print(f"Cantidad de gente con email: {with_email}, eso es el {((with_email*100)/total_rows):.2f}% de los usuarios")

# Vamos a ver el número de cuentas sin cursos
no_courses = df.query("courses == '[]'").shape[0]
print(f"Cantidad de gente sin cursos: {no_courses}, eso es el {((no_courses*100)/total_rows):.2f}% de los usuarios")

(879, 7)
Cantidad de gente con foto: 65, eso es el 7.39% de los usuarios
Cantidad de gente con email: 25, eso es el 2.84% de los usuarios
Cantidad de gente sin cursos: 17, eso es el 1.93% de los usuarios


Quiero mirar estadísticas de conexión, para eso tengo que formatear las columnas con fechas a algo legible para python:

In [3]:
import re

# Diccionario de meses españoles para convertir a número
meses = {
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
    "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
    "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
}

# Definimos una funcion auxiliar
def parse_fecha_hora(texto):
    if pd.isna(texto) or texto in ["N/A", "Nunca", ""]: # Si no podemos determinar la fecha marcamos Not a Time para pandas
        return pd.NaT
    # Regex para día, mes, año, hora:minuto
    target = re.search(r"(\d{1,2}) de (\w+) de (\d{4}), (\d{1,2}:\d{2})", texto)
    if target:
        dia, mes_texto, año, hora_minuto = target.groups() # Pasar a diferentes variables
        mes = meses.get(mes_texto.lower()) # Pasar el mes a minusculas por si acaso y sacar el numero de mes de nuetro diccionario
        if mes:
            fecha_str = f"{año}-{mes}-{int(dia):02d} {hora_minuto}" # Reescribir la fecha sin los "de" y con todo formateado
            return pd.to_datetime(fecha_str, format="%Y-%m-%d %H:%M") # Devolver con formato Datetime de pandas
    return pd.NaT

df["create_date"] = df["create_date"].apply(parse_fecha_hora) # Aplicamos la funcion a la columna con .apply
df["last_conn"] = df["last_conn"].apply(parse_fecha_hora)

print(df.head(3).to_string(index=False))

 id       username                                                                  profile_pic         create_date           last_conn    email courses
  1     Gonbidatua                                                                     No photo                 NaT                 NaT No email      []
  2 Admin ICJardin                                                                     No photo 2024-07-05 09:55:00 2025-10-07 10:12:00 No email      []
  3     Admin User https://moodle.icjardin.com/pluginfile.php/18/user/icon/boost/f1?rev=2622497 2020-11-17 09:45:00 2024-06-27 19:45:00 No email      []


In [4]:
from dateutil.relativedelta import relativedelta # Libreria de deltas, un delta es la diferencia entre dos puntos (temporales en este caso)


# Quiero ver el primer usuario creado y con acceso
oldest_created = df.sort_values("create_date").iloc[0]
print(oldest_created["create_date"]) # Filtro la columna por no enseñar informacion personal

rd = relativedelta(pd.Timestamp.now(), oldest_created["create_date"])
print(f"{rd.years} años, {rd.months} meses, {rd.days} días")

2009-12-02 19:28:00
15 años, 10 meses, 4 días


Quiero también sacar una lista de cuantos cursos hay, y un ranking de cursos por participantes:

In [5]:
import ast # Libreria para evaluar literales, es para que el string "['algo']" se evalue como una lista python [algo]

# Convertir strings a listas reales en el dataframe
df["courses"] = df["courses"].apply(lambda x: ast.literal_eval(x) if x else []) # La lambda hace que si no detecta nada cree la lista vacia

# Flatten y sacar distinct
all_courses = [curso.strip().lower() for sublist in df["courses"] for curso in sublist if curso] # Teniendo listas podemos usarlas como un objeto de python e iterar
distinct_courses = set(all_courses) # El objeto set se asegura que solo haya un elemento de cada
print("Número de cursos distintos:", len(distinct_courses))

Número de cursos distintos: 456


Con el poder de GPT podemos sacar un script que cree una red visual de nodos que representan cada clase, tarda un rato al ser 456 fotos y no lo he dejado completo:

In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import ast
import os

# ======== Leer Excel ========
df = pd.read_excel("output.xlsx")

# Si la columna 'courses' está en texto tipo "['Math', 'Physics']"
if isinstance(df["courses"].iloc[0], str):
    df["courses"] = df["courses"].apply(ast.literal_eval)

# Crear carpeta para las capturas
os.makedirs("graficos_cursos", exist_ok=True)

# ======== Recorrer todos los cursos únicos ========
todos_los_cursos = sorted({curso for lista in df["courses"] for curso in lista})
print(f"Se generarán {len(todos_los_cursos)} gráficos...")

for curso in todos_los_cursos:
    # Filtrar personas inscritas en este curso
    df_filtrado = df[df["courses"].apply(lambda c: curso in c)]

    if df_filtrado.empty:
        continue

    # Crear grafo solo con las conexiones de este curso
    G = nx.Graph()
    for _, row in df_filtrado.iterrows():
        persona = row["username"]
        G.add_edge(persona, curso)

    # Calcular posiciones
    pos = nx.spring_layout(G, seed=42)

    # Aristas
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color="#aaa"),
        hoverinfo="none",
        mode="lines"
    )

    # Nodos
    node_x, node_y, node_text, node_color = [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_color.append("#ff7f0e" if node == curso else "#1f77b4")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers+text",
        text=node_text,
        textposition="top center",
        hoverinfo="text",
        marker=dict(size=18, color=node_color, line_width=2)
    )

    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            title=f"Red de alumnos del curso '{curso}'",
            showlegend=False,
            hovermode="closest",
            margin=dict(b=0, l=0, r=0, t=40),
            xaxis=dict(showgrid=False, zeroline=False, visible=False),
            yaxis=dict(showgrid=False, zeroline=False, visible=False)
        )
    )

    # ======== Guardar PNG ========
    nombre_archivo = f"graficos_cursos/{curso.replace('/', '_').replace(' ', '_')}.png"
    fig.write_image(nombre_archivo, width=1000, height=800, scale=2)
    print(f"✅ Guardado: {nombre_archivo}")

print("\n🎨 Capturas generadas en la carpeta: graficos_cursos/")


Podemos ver si dos personas tienen cursos en comun:

In [ ]:
import ast

def cursos_en_comun(df, alumno1, alumno2):
    """
    Devuelve los cursos compartidos entre dos alumnos, si existen.
    df: DataFrame con columnas 'username' y 'courses'
    alumno1, alumno2: nombres de usuario a comparar
    """
    # Asegurar que la columna de cursos sea lista
    if isinstance(df["courses"].iloc[0], str):
        df["courses"] = df["courses"].apply(ast.literal_eval)

    # Buscar las filas de los alumnos
    c1 = df.loc[df["username"] == alumno1, "courses"]
    c2 = df.loc[df["username"] == alumno2, "courses"]

    if c1.empty or c2.empty:
        return f"❌ Uno o ambos alumnos no existen en el DataFrame."

    # Crear sets
    cursos1 = set(c1.iloc[0])
    cursos2 = set(c2.iloc[0])

    comun = cursos1 & cursos2  # intersección de sets cursos1 ∩ cursos2

    if comun:
        return f"✅ {alumno1} y {alumno2} comparten estos cursos: {', '.join(comun)}"
    else:
        return f"🚫 {alumno1} y {alumno2} no comparten ningún curso."


print(cursos_en_comun(df,"Nombre1","Nombre2"))

❌ Uno o ambos alumnos no existen en el DataFrame.


Y podemos ver si alguien puede conocer a alguien a través de conocidos de clases, por usar el grafo más:

In [10]:
import pandas as pd
import networkx as nx
import ast

# Leer Excel
df = pd.read_excel("output.xlsx")
if isinstance(df["courses"].iloc[0], str):
    df["courses"] = df["courses"].apply(ast.literal_eval)

# Crear grafo bipartito: alumnos <-> cursos
G = nx.Graph()
for _, row in df.iterrows():
    usuario = row["username"]
    for curso in row["courses"]:
        G.add_edge(usuario, curso)

# Función para encontrar camino y mostrarlo tipo "Alba -> Fisica -> Belen -> Ingles -> Carlos"
def camino_completo(grafo, usuario1, usuario2):
    try:
        # Encuentra un camino más corto
        path = nx.shortest_path(grafo, source=usuario1, target=usuario2)
    except nx.NetworkXNoPath:
        return f"🚫 No hay camino entre {usuario} y {usuario2}"

    # Construir cadena indicando si es alumno o curso
    cadena = []
    for i in range(len(path)-1):
        cadena.append(path[i])

    cadena.append(path[-1])
    
    # Añadir flechas
    return " -> ".join(cadena)

# Ejemplo
print(camino_completo(G, "1", "2"))


NodeNotFound: Source 1 is not in G

Vamos a simplemente ver ranking de cursos por cantidad de usuarios inscritos:

In [21]:
import pandas as pd
import ast

# Leer Excel
df = pd.read_excel("output.xlsx")

# Si la columna 'courses' es texto tipo "['Math', 'Physics']"
if isinstance(df["courses"].iloc[0], str):
    df["courses"] = df["courses"].apply(ast.literal_eval)

# Explode para tener 1 fila por alumno-curso
df_exploded = df.explode("courses")

# Contar alumnos por curso
conteo = df_exploded.groupby("courses")["username"].nunique().reset_index()
conteo = conteo.rename(columns={"username": "cantidad_usuarios"})

# Ordenar de mayor a menor
conteo = conteo.sort_values("cantidad_usuarios", ascending=False).reset_index(drop=True)

print(conteo.head(1))

                courses  cantidad_usuarios
0  Jefatura de Estudios                160


Hay muchos cursos vaciados, vamos a suponer que los alumnos graduados se borran y solo el profesor se queda en el curso hasta que se elimina este o son proyectos o asignaturas sin abrir al alumnado:

In [18]:
# Filtrar cursos con solo 1 usuario
cursos_solo_un_usuario = conteo[conteo["cantidad_usuarios"] == 1]

# Cantidad de cursos con un solo usuario
cantidad = len(cursos_solo_un_usuario)

print(f"Hay {cantidad} cursos con solo 1 usuario.")

Hay 83 cursos con solo 1 usuario.


Se pueden hacer demasiadas cosas a partir de un scraping y un excel cargado de datos